In [209]:
import pandas as pd 
import numpy as np
from sklearn import tree
import graphviz
import os
from sklearn.metrics import roc_auc_score
from scipy.stats import ks_2samp
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn import metrics
os.environ["PATH"]+=os.pathsep+'/opt/homebrew/Cellar/graphviz/12.0.0/bin/'
df=pd.read_csv('L26_new_data_60k_202407_v3.csv')
# df.info() ##查看数据多少列，数据大小

# Indicators

In [317]:
# approvel rate
df_pivot=df[df['control_group']==0].groupby("order_date").agg(order_cnt=("order_id", "size"),
                         approval_cnt=("order_status", "sum"),).reset_index()
df_pivot['approvel_rate']=df_pivot['approval_cnt']/df_pivot['order_cnt']
df_pivot

,order_date,order_cnt,approval_cnt,approvel_rate
0,2024/7/16,5195,3016,0.580558
1,2024/7/17,5511,3147,0.571040
2,2024/7/18,5516,3178,0.576142
3,2024/7/19,5285,3016,0.570672
4,2024/7/20,5392,3071,0.569547
5,2024/7/21,5701,3251,0.570251
6,2024/7/22,5402,3061,0.566642
7,2024/7/23,5295,3018,0.569972
8,2024/7/24,5553,3216,0.579146
9,2024/7/25,6110,2938,0.480851


In [319]:
#Coverage Rate
df['s1']=np.where(df.fico_score<600,1,0)
df['s2']=np.where((df.fico_score<650)&(df.occupation_new=='parttime'),1,0)

df_pivot=df[df['control_group']==0].groupby("order_date").agg(order_cnt=("order_id", "size"),
                         approval_cnt=("order_status", "sum"),
                         s1_cnt=("s1", "sum"),
                         s2_cnt=("s2", "sum")).reset_index()
df_pivot['approvel_rate']=df_pivot['approval_cnt']/df_pivot['order_cnt']
df_pivot['s1_rate']=df_pivot['s1_cnt']/df_pivot['order_cnt']
df_pivot['s2_rate']=df_pivot['s2_cnt']/df_pivot['order_cnt']
df_pivot

,order_date,order_cnt,approval_cnt,s1_cnt,s2_cnt,approvel_rate,s1_rate,s2_rate
0,2024/7/16,5195,3016,2064,443,0.580558,0.397305,0.085274
1,2024/7/17,5511,3147,2245,443,0.571040,0.407367,0.080385
2,2024/7/18,5516,3178,2221,420,0.576142,0.402647,0.076142
3,2024/7/19,5285,3016,2173,427,0.570672,0.411164,0.080795
4,2024/7/20,5392,3071,2205,427,0.569547,0.408939,0.079191
5,2024/7/21,5701,3251,2335,444,0.570251,0.409577,0.077881
6,2024/7/22,5402,3061,2242,430,0.566642,0.415031,0.079600
7,2024/7/23,5295,3018,2171,430,0.569972,0.410009,0.081209
8,2024/7/24,5553,3216,2215,456,0.579146,0.398883,0.082118
9,2024/7/25,6110,2938,2831,1354,0.480851,0.463339,0.221604


In [327]:
df['s2_2']=np.where((df.occupation_new=='parttime'),1,0)
df['s2_1']=np.where((df.fico_score<650)&(df.fico_score>=600),1,0)
df_pivot=df[df['control_group']==0].groupby("order_date").agg(order_cnt=("order_id", "size"),
                         approval_cnt=("order_status", "sum"),
                         s1_cnt=("s1", "sum"),
                         s2_cnt=("s2", "sum"),
                         s22_cnt=("s2_2", "sum"),
                         s21_cnt=("s2_1", "sum")).reset_index()
df_pivot['approvel_rate']=df_pivot['approval_cnt']/df_pivot['order_cnt']
df_pivot['s1_rate']=df_pivot['s1_cnt']/df_pivot['order_cnt']
df_pivot['s2_rate']=df_pivot['s2_cnt']/df_pivot['order_cnt']
df_pivot['s21_rate']=df_pivot['s21_cnt']/df_pivot['order_cnt']
df_pivot['s22_rate']=df_pivot['s22_cnt']/df_pivot['order_cnt']
df_pivot

,order_date,order_cnt,approval_cnt,s1_cnt,s2_cnt,s22_cnt,s21_cnt,approvel_rate,s1_rate,s2_rate,s21_rate,s22_rate
0,2024/7/16,5195,3016,2064,443,762,776,0.580558,0.397305,0.085274,0.149374,0.146679
1,2024/7/17,5511,3147,2245,443,783,752,0.571040,0.407367,0.080385,0.136454,0.142079
2,2024/7/18,5516,3178,2221,420,775,849,0.576142,0.402647,0.076142,0.153916,0.140500
3,2024/7/19,5285,3016,2173,427,776,752,0.570672,0.411164,0.080795,0.142289,0.146831
4,2024/7/20,5392,3071,2205,427,782,761,0.569547,0.408939,0.079191,0.141135,0.145030
5,2024/7/21,5701,3251,2335,444,805,832,0.570251,0.409577,0.077881,0.145939,0.141203
6,2024/7/22,5402,3061,2242,430,779,756,0.566642,0.415031,0.079600,0.139948,0.144206
7,2024/7/23,5295,3018,2171,430,791,757,0.569972,0.410009,0.081209,0.142965,0.149386
8,2024/7/24,5553,3216,2215,456,824,808,0.579146,0.398883,0.082118,0.145507,0.148388
9,2024/7/25,6110,2938,2831,1354,1680,941,0.480851,0.463339,0.221604,0.154010,0.274959


In [343]:
#计算PSI 
df=pd.read_csv('L26_new_data_60k_202407_v3.csv')
df=df[df['control_group']==1]
new_date=22
old_date=new_date-1
def calculate_psi(current_distribution, reference_distribution):
    n_bins = len(current_distribution)
    total_count = sum(current_distribution) + sum(reference_distribution)
    def psi_bin(bin_index):
        actual = current_distribution[bin_index] / total_count
        expected = reference_distribution[bin_index] / total_count
        if expected == 0:
            expected = 0.0001  # 防止除以零错误
        return actual * abs(np.log(actual / expected))
    psi_values = [psi_bin(i) for i in range(n_bins)]
    return np.sum(psi_values)

df['order_date'] = pd.to_datetime(df['order_date'])  # 确保order_date是datetime类型
df_2009 = df[df['order_date'].dt.day== old_date]
df_2010 = df[df['order_date'].dt.day== new_date]
# 为了演示，我们假设一个简单的分箱过程，分5箱
bins = pd.cut(df['fico_score'], bins=10, labels=False, include_lowest=True)  # 5个等宽分箱
dist_2009 = df_2009.groupby(bins).size() / len(df_2009)  # 计算各分箱的频率
dist_2010 = df_2010.groupby(bins).size() / len(df_2010)  # 同上
psi_value = calculate_psi(dist_2010, dist_2009)  # 注意顺序，一般是新分布对老分布
print(new_date,f"日PSI值为: {psi_value}")

22 日PSI值为: 0.05875158154414014


In [361]:
#计算PSI-简单循环
def calculate_psi(current_distribution, reference_distribution):
    n_bins = len(current_distribution)
    total_count = sum(current_distribution) + sum(reference_distribution)
    def psi_bin(bin_index):
        actual = current_distribution[bin_index] / total_count
        expected = reference_distribution[bin_index] / total_count
        if expected == 0:
            expected = 0.0001  # 防止除以零错误
        return actual * abs(np.log(actual / expected))
    psi_values = [psi_bin(i) for i in range(n_bins)]
    return np.sum(psi_values)
#
df=pd.read_csv('L26_new_data_60k_202407_v3.csv')
df=df[df['control_group']==0]
df['order_date'] = pd.to_datetime(df['order_date'])  # 确保order_date是datetime类型

for new_date in range(17,26):
    old_date=new_date-1
    df_2009 = df[df['order_date'].dt.day== old_date]
    df_2010 = df[df['order_date'].dt.day== new_date]
    # 为了演示，我们假设一个简单的分箱过程，分5箱
    bins = pd.cut(df['fico_score'], bins=5, labels=False, include_lowest=True)  # 5个等宽分箱
    dist_2009 = df_2009.groupby(bins).size() / len(df_2009)  # 计算各分箱的频率
    dist_2010 = df_2010.groupby(bins).size() / len(df_2010)  # 同上
    psi_value = calculate_psi(dist_2010, dist_2009)  # 注意顺序，一般是新分布对老分布
    print(new_date,f"对比前日PSI值为: {psi_value}")

17 对比前日PSI值为: 0.020435275175123623
18 对比前日PSI值为: 0.025858932937528122
19 对比前日PSI值为: 0.022233854495948592
20 对比前日PSI值为: 0.017886254757142325
21 对比前日PSI值为: 0.01609629676388693
22 对比前日PSI值为: 0.014932640214631407
23 对比前日PSI值为: 0.014847599567396275
24 对比前日PSI值为: 0.015130630484796204
25 对比前日PSI值为: 0.06963102237007923


# Is strategy s2 still meeting the threshold of  deployment(2-lift in mob3_m2)

In [246]:
#方法1
df=pd.read_csv('L26_new_data_60k_202407_v3.csv')
df['s1']=np.where(df.fico_score<600,1,0)
df['s2']=np.where((df.fico_score<650)&(df.occupation_new=='parttime'),1,0)
df_pivot=df[df['control_group']==1].groupby("s2").agg(order_cnt=("order_id", "size"),
                         approval_cnt=("order_status", "sum"),
                         fpd30=("fpd30", "sum"),
                         mob3_m2=("mob3_m2", "sum"),
                         principal=("principal", "sum")).reset_index()
df_pivot['fpd30_rate']=df_pivot['fpd30']/df_pivot['approval_cnt']
df_pivot['mob3_m2_rate']=df_pivot['mob3_m2']/df_pivot['approval_cnt']
df_pivot

,s2,order_cnt,approval_cnt,fpd30,mob3_m2,principal,fpd30_rate,mob3_m2_rate
0,0,5544,5544,198,463,18769000,0.035714,0.083514
1,1,496,496,54,114,1561100,0.108871,0.229839


In [366]:
df=pd.read_csv('L26_new_data_60k_202407_v3.csv')
df_pivot=df.groupby("control_group").agg(order_cnt=("order_id", "size"),
                         approval_cnt=("order_status", "sum"),
                         fpd30=("fpd30", "sum"),
                         mob3_m2=("mob3_m2", "sum"),
                         principal=("principal", "sum")).reset_index()
df_pivot['fpd30_rate']=df_pivot['fpd30']/df_pivot['approval_cnt']
df_pivot['mob3_m2_rate']=df_pivot['mob3_m2']/df_pivot['approval_cnt']
df_pivot

,control_group,order_cnt,approval_cnt,fpd30,mob3_m2,principal,fpd30_rate,mob3_m2_rate
0,0,54960,30912,247,708,105205000,0.007990,0.022904
1,1,6040,6040,252,577,20330100,0.041722,0.095530


In [390]:
#方法2
df=pd.read_csv('L26_new_data_60k_202407_v3.csv')
df['s1']=np.where(df.fico_score<600,1,0)
df['s2']=np.where((df.fico_score<650)&(df.occupation_new=='parttime'),1,0)
df['s2_margin']=np.where((df.s1==0)&(df.s2==1),1,0)#0.0367--in out
df['s1_margin']=np.where((df.s1==1)&(df.s2==0),1,0)#0.176 --out in
df['s12']=np.where((df.s1==1)&(df.s2==1),1,0)#0.28 --out out
df['s12_verse']=np.where((df.s1==0)&(df.s2==0),1,0)#0.027 --in in

df_pivot=df[df['control_group']==1].groupby("s1_margin").agg(order_cnt=("order_id", "size"),
                         approval_cnt=("order_status", "sum"),
                         fpd30=("fpd30", "sum"),
                         mob3_m2=("mob3_m2", "sum"),
                         principal=("principal", "sum")).reset_index()
df_pivot['fpd30_rate']=df_pivot['fpd30']/df_pivot['approval_cnt']
df_pivot['mob3_m2_rate']=df_pivot['mob3_m2']/df_pivot['approval_cnt']
df_pivot

,s1_margin,order_cnt,approval_cnt,fpd30,mob3_m2,principal,fpd30_rate,mob3_m2_rate
0,0,3951,3951,81,210,13114900,0.020501,0.053151
1,1,2089,2089,171,367,7215200,0.081857,0.175682


# Swap in Swap out

In [295]:
#control_group=1
df=pd.read_csv('L26_new_data_60k_202407_v3.csv')
df['s1']=np.where(df.fico_score<600,1,0)
df['s2']=np.where((df.fico_score<650)&(df.occupation_new=='parttime'),1,0)
df['s3']=np.where((df.overdue_times_in_1year>5)&(df.vintage_score<760),1,0)
df['s2_margin']=np.where((df.s1==0)&(df.s2==1),1,0)
df['s3_margin']=np.where((df.s1==0)&(df.s3==1),1,0)

df_pivot=df[df['control_group']==1].groupby("s3_margin").agg(order_cnt=("order_id", "size"),
                         approval_cnt=("order_status", "sum"),
                         fpd30=("fpd30", "sum"),
                         mob3_m2=("mob3_m2", "sum"),
                         principal=("principal", "sum")).reset_index()
df_pivot['fpd30_rate']=df_pivot['fpd30']/df_pivot['approval_cnt']
df_pivot['mob3_m2_rate']=df_pivot['mob3_m2']/df_pivot['approval_cnt']
df_pivot

,s3_margin,order_cnt,approval_cnt,fpd30,mob3_m2,principal,fpd30_rate,mob3_m2_rate
0,0,5998,5998,248,569,20177900,0.041347,0.094865
1,1,42,42,4,8,152200,0.095238,0.190476


In [312]:
#swap in out
df=pd.read_csv('L26_new_data_60k_202407_v3.csv')
df['s1']=np.where(df.fico_score<600,1,0)
df['s2']=np.where((df.fico_score<650)&(df.occupation_new=='parttime'),1,0)
df['s3']=np.where((df.overdue_times_in_1year>5)&(df.vintage_score<760),1,0)
df['s2_margin']=np.where((df.s1==0)&(df.s2==1),1,0)
df['s3_margin']=np.where((df.s1==0)&(df.s3==1),1,0)
df['in_out']=np.where((df.s2_margin==0)&(df.s3_margin==1),1,0)
df['out_in']=np.where((df.s2_margin==1)&(df.s3_margin==0),1,0)

df_pivot=df[df['control_group']==1].groupby("out_in").agg(order_cnt=("order_id", "size"),
                         approval_cnt=("order_status", "sum"),
                         fpd30=("fpd30", "sum"),
                         mob3_m2=("mob3_m2", "sum"),
                         principal=("principal", "sum")).reset_index()
df_pivot['fpd30_rate']=df_pivot['fpd30']/df_pivot['approval_cnt']
df_pivot['mob3_m2_rate']=df_pivot['mob3_m2']/df_pivot['approval_cnt']
df_pivot

,out_in,order_cnt,approval_cnt,fpd30,mob3_m2,principal,fpd30_rate,mob3_m2_rate
0,0,5935,5935,252,573,19996700,0.04246,0.096546
1,1,105,105,0,4,333400,0.00000,0.038095


In [262]:
df=pd.read_csv('L26_new_data_60k_202407_v3.csv')
df[df['control_group']==1].describe()

,Unnamed: 0,order_id,user_id,age,fico_score,vintage_score,medical_records_12m,income,principal,term,...,order_status,fpd30,mob3_m2,apply_amount,control_group,credit_card_utilization,credit_inquires_2years,overdue_times_in_1year,credit_account_avg_mth,number_of_credit_accounts
count,6040.000000,6.040000e+03,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000,6040.0,...,6040.0,6040.000000,6040.00000,6040.000000,6040.0,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000
mean,30619.966722,1.002494e+07,114854.158940,42.787881,626.742384,743.388907,4.226821,34247.019868,3365.910596,12.0,...,1.0,0.041722,0.09553,3365.910596,1.0,0.347263,7.695695,2.263079,30.691060,4.886093
std,17596.541834,1.448214e+04,14482.144174,14.266076,107.194959,144.765312,2.971278,10017.896276,1919.283231,0.0,...,0.0,0.199969,0.29397,1919.283231,0.0,0.199739,5.883743,1.631825,16.629106,2.871644
min,10.000000,1.000001e+07,89920.000000,18.000000,400.000000,500.000000,0.000000,1000.000000,100.000000,12.0,...,1.0,0.000000,0.00000,100.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.000000
25%,15325.750000,1.001248e+07,102397.500000,30.600000,544.000000,620.000000,2.000000,28000.000000,1800.000000,12.0,...,1.0,0.000000,0.00000,1800.000000,1.0,0.180000,3.000000,1.000000,18.000000,3.000000
50%,30764.000000,1.002494e+07,114850.000000,42.900000,634.000000,743.000000,4.000000,34000.000000,3300.000000,12.0,...,1.0,0.000000,0.00000,3300.000000,1.0,0.350000,6.000000,2.000000,31.000000,4.000000
75%,45795.250000,1.003719e+07,127105.000000,54.800000,722.000000,871.250000,6.000000,41000.000000,4700.000000,12.0,...,1.0,0.000000,0.00000,4700.000000,1.0,0.520000,11.000000,3.000000,43.000000,6.000000
max,60990.000000,1.004999e+07,139900.000000,67.900000,799.000000,996.000000,16.000000,70000.000000,10200.000000,12.0,...,1.0,1.000000,1.00000,10200.000000,1.0,0.700000,37.000000,9.000000,82.000000,18.000000


Unnamed: 0                   0
order_id                     0
user_id                      0
order_date                   0
age                          0
fico_score                   0
vintage_score                0
ethnicity                    0
medical_records_12m          0
income                       0
principal                    0
term                         0
interest                     0
order_status                 0
fpd30                        0
mob3_m2                      0
apply_amount                 0
control_group                0
occupation                   0
credit_card_utilization      0
credit_inquires_2years       0
overdue_times_in_1year       0
credit_account_avg_mth       0
number_of_credit_accounts    0
education                    0
s1                           0
s2                           0
dtype: int64

Unnamed: 0                   0
order_id                     0
user_id                      0
order_date                   0
age                          0
fico_score                   0
vintage_score                0
ethnicity                    0
medical_records_12m          0
income                       0
principal                    0
term                         0
interest                     0
order_status                 0
fpd30                        0
mob3_m2                      0
apply_amount                 0
control_group                0
occupation                   0
credit_card_utilization      0
credit_inquires_2years       0
overdue_times_in_1year       0
credit_account_avg_mth       0
number_of_credit_accounts    0
education                    0
s1                           0
s2                           0
dtype: int64